### 1 - Import test data

In [1]:
import pandas as pd
import pandas_ta as ta

df = pd.read_csv('downloads/mt/EURUSD_M5_202205090005_202209302350.csv', sep="\\t")
df['Gmt time'] = df['<DATE>'] + ' ' + df['<TIME>']
df.rename(columns={'<CLOSE>' : 'Close', '<HIGH>': 'High', '<OPEN>' : 'Open', '<LOW>':'Low'}, inplace=True)
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%Y.%m.%d %H:%M:%S')
df=df[df.High!=df.Low]
df.set_index("Gmt time", inplace=True)

C:\Users\S.Eyvazi\AppData\Local\Temp\ipykernel_3900\3633356274.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  df = pd.read_csv('downloads/mt/EURUSD_M5_202205090005_202209302350.csv', sep="\\t")


In [2]:
df["EMA_slow"]=ta.ema(df.Close, length=50)
df["EMA_fast"]=ta.ema(df.Close, length=30)
df['RSI']=ta.rsi(df.Close, length=10)
my_bbands = ta.bbands(df.Close, length=15, std=1.5)
df['ATR']=ta.atr(df.High, df.Low, df.Close, length=7)
df=df.join(my_bbands)


In [3]:
df.head()

,<DATE>,<TIME>,Open,High,Low,Close,<TICKVOL>,<VOL>,<SPREAD>,EMA_slow,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5
Gmt time,,,,,,,,,,,,,,,,,,
2022-05-09 00:05:00,2022.05.09,00:05:00,1.05400,1.05425,1.05395,1.05407,34,0,52,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-09 00:10:00,2022.05.09,00:10:00,1.05407,1.05425,1.05403,1.05417,20,0,65,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-09 00:15:00,2022.05.09,00:15:00,1.05418,1.05424,1.05389,1.05420,37,0,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-09 00:20:00,2022.05.09,00:20:00,1.05390,1.05424,1.05385,1.05420,26,0,25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-05-09 00:25:00,2022.05.09,00:25:00,1.05391,1.05421,1.05387,1.05399,32,0,32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
def ema_signal(df, current_candle, backcandles):
    df_slice = df.reset_index().copy()
    # Get the range of candles to consider
    start = max(0, current_candle - backcandles)
    end = current_candle
    relevant_rows = df_slice.iloc[start:end]

    # Check if all EMA_fast values are below EMA_slow values
    if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
        return 1
    elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
        return 2
    else:
        return 0



from tqdm import tqdm
tqdm.pandas()
df.reset_index(inplace=True)
df['EMASignal'] = df.progress_apply(lambda row: ema_signal(df, row.name, 7) , axis=1) #if row.name >= 20 else 0

100%|██████████| 30207/30207 [02:14<00:00, 224.60it/s]


In [5]:
def total_signal(df, current_candle, backcandles):
    if (ema_signal(df, current_candle, backcandles)==2
        and df.Close[current_candle]<=df['BBL_15_1.5'][current_candle]
        #and df.RSI[current_candle]<60
        ):
            return 2
    if (ema_signal(df, current_candle, backcandles)==1
        and df.Close[current_candle]>=df['BBU_15_1.5'][current_candle]
        #and df.RSI[current_candle]>40
        ):
    
            return 1
    return 0
        
df['TotalSignal'] = df.progress_apply(lambda row: total_signal(df, row.name, 7), axis=1)

100%|██████████| 30207/30207 [04:55<00:00, 102.32it/s]


In [12]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime
st=100
dfpl = df[st:st+350]
#dfpl.reset_index(inplace=True)
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['Open'],
                high=dfpl['High'],
                low=dfpl['Low'],
                close=dfpl['Close']),

                go.Scatter(x=dfpl.index, y=dfpl['BBL_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBL"),
                go.Scatter(x=dfpl.index, y=dfpl['BBU_15_1.5'], 
                           line=dict(color='green', width=1), 
                           name="BBU"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_fast'], 
                           line=dict(color='black', width=1), 
                           name="EMA_fast"),
                go.Scatter(x=dfpl.index, y=dfpl['EMA_slow'], 
                           line=dict(color='blue', width=1), 
                           name="EMA_slow")])

fig.add_scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                marker=dict(size=5, color="MediumPurple"),
                name="entry")

fig.show()

KeyError: 'pointpos'

In [6]:
def SIGNAL():
    return df.TotalSignal

In [88]:
from backtesting import Strategy
from backtesting import Backtest

class MyStrat(Strategy):
    mysize = 3000
    slcoef = 1.894736
    TPSLRatio = 1.05263
    rsi_length = 16

    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)
        #df['RSI']=ta.rsi(df.Close, length=self.rsi_length)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        if self.signal1==2 and len(self.trades)==0:
            
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

bt = Backtest(df, MyStrat, cash=250, margin=1/30)
bt.run()
stats = bt.run()

C:\Users\S.Eyvazi\AppData\Local\Temp\ipykernel_3900\1747181637.py:31: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(df, MyStrat, cash=250, margin=1/30)


In [89]:
stats._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-3000,89,95,1.05157,1.050779,2.371882,0.000752,89,95,6
1,-3000,108,116,1.05132,1.050431,2.668435,0.000846,108,116,8
2,-3000,122,141,1.05168,1.050195,4.453956,0.001412,122,141,19
3,3000,193,199,1.05501,1.053599,-4.234325,-0.001338,193,199,6
4,3000,203,224,1.05323,1.054895,4.994198,0.001581,203,224,21
...,...,...,...,...,...,...,...,...,...,...
1068,3000,30076,30079,0.97974,0.977872,-5.604353,-0.001907,30076,30079,3
1069,-3000,30106,30106,0.97652,0.974266,6.762704,0.002308,30106,30106,0
1070,-3000,30109,30112,0.97750,0.974265,9.703918,0.003309,30109,30112,3
1071,-3000,30124,30133,0.97746,0.979815,-7.065629,-0.002410,30124,30133,9


In [129]:
df[df.index == 87 ]
# df.query('Close == 1.05501' ).head()

,Gmt time,<DATE>,<TIME>,Open,High,Low,Close,<TICKVOL>,<VOL>,<SPREAD>,...,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal
87,2022-05-09 07:20:00,2022.05.09,07:20:00,1.05105,1.05127,1.05101,1.05119,183,0,0,...,1.051211,54.219837,0.000387,1.050537,1.050872,1.051207,0.06376,0.974604,1,0


In [91]:
bt.run()

Start                                     0.0
End                                   30206.0
Duration                              30206.0
Exposure Time [%]                   47.055318
Equity Final [$]                   393.801495
Equity Peak [$]                    438.029829
Return [%]                          57.520598
Buy & Hold Return [%]               -7.036535
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -18.280975
Avg. Drawdown [%]                   -1.781074
Max. Drawdown Duration                 5322.0
Avg. Drawdown Duration             234.056911
# Trades                               1073.0
Win Rate [%]                        52.469711
Best Trade [%]                       0.463591
Worst Trade [%]                     -0.428693
Avg. Trade [%]                    

In [3]:
import pandas as pd
import pandas_ta as ta

df = pd.read_csv('clean15.csv')
df.rename(columns={'datetime' : 'Gmt time', 'close' : 'Close', 'high': 'High', 'open' : 'Open', 'low':'Low'}, inplace=True)
# df = pd.read_csv("EURUSD_Candlestick_5_M_ASK_30.09.2019-30.09.2022.csv")
df["Gmt time"]=df["Gmt time"].str.replace(".000","")
df['Gmt time']=pd.to_datetime(df['Gmt time'],format='%Y-%m-%d %H:%M:%S')
df=df[df.High!=df.Low]
df.set_index("Gmt time", inplace=True)




from tqdm import tqdm
class ScalpingStrategyBasedEMABollingerBand:
    
    def __init__(self, df : pd.DataFrame, ema_slow_length : int, ema_fast_length : int, back_candle_no:int, last_candle_num_to_processing:int):
        self.ema_slow_length = ema_slow_length 
        self.ema_fast_length = ema_fast_length
        self.back_candle_no = back_candle_no
        self.last_candle_num_to_processing = last_candle_num_to_processing
        self.df = df[-last_candle_num_to_processing:-1]
        
        self.df["EMA_slow"]=ta.ema(self.df.Close, length=self.ema_slow_length)
        self.df["EMA_fast"]=ta.ema(self.df.Close, length=self.ema_fast_length)
        self.df['RSI']=ta.rsi(self.df.Close, length=10)
        my_bbands = ta.bbands(self.df.Close, length=15, std=1.5)
        self.df['ATR']=ta.atr(self.df.High, self.df.Low, self.df.Close, length=7)
        self.df=self.df.join(my_bbands)
        self.calculate_ema_signal()
 

    def ema_signal(self,current_candle):
        df_slice = self.df.reset_index().copy()
        # Get the range of candles to consider
        start = max(0, current_candle - self.back_candle_no)
        end = current_candle
        relevant_rows = df_slice.iloc[start:end]
    
        # Check if all EMA_fast values are below EMA_slow values
        if all(relevant_rows["EMA_fast"] < relevant_rows["EMA_slow"]):
            return 1
        elif all(relevant_rows["EMA_fast"] > relevant_rows["EMA_slow"]):
            return 2
        else:
            return 0

    def calculate_ema_signal(self):
        tqdm.pandas()
        self.df.reset_index(inplace=True)
        self.df['EMASignal'] = self.df.progress_apply(lambda row: self.ema_signal(row.name) , axis=1) #if row.name >= 20 else 0

    def total_signal(self,current_candle):
        if (self.ema_signal(current_candle)==2
                and self.df.Close[current_candle]<=self.df['BBL_15_1.5'][current_candle]
        ):
            return 2
        
        if (self.ema_signal(current_candle)==1
                and self.df.Close[current_candle]>=self.df['BBU_15_1.5'][current_candle]
        ):
            return 1
        
        return 0
    
    def calc_total_signal(self):
        self.df['TotalSignal'] = self.df.progress_apply(lambda row: self.total_signal(row.name), axis=1)
        
scalping = ScalpingStrategyBasedEMABollingerBand(df,30,50, 10, 30000)
scalping.calc_total_signal()

C:\Users\S.Eyvazi\AppData\Local\Temp\ipykernel_22660\3161368335.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["EMA_slow"]=ta.ema(self.df.Close, length=self.ema_slow_length)
C:\Users\S.Eyvazi\AppData\Local\Temp\ipykernel_22660\3161368335.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.df["EMA_fast"]=ta.ema(self.df.Close, length=self.ema_fast_length)
C:\Users\S.Eyvazi\AppData\Local\Temp\ipykernel_22660\3161368335.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [42]:
def SIGNAL():
    return scalping.df.TotalSignal

class MyStrat(Strategy):
    mysize = 3000
    slcoef = 1.894736
    TPSLRatio = 1.05263
    rsi_length = 16

    def init(self):
        super().init()
        self.signal1 = self.I(SIGNAL)

    def next(self):
        super().next()
        slatr = self.slcoef*self.data.ATR[-1]
        TPSLRatio = self.TPSLRatio

        if self.signal1==2 and len(self.trades)==0:
            sl1 = self.data.Close[-1] - slatr
            tp1 = self.data.Close[-1] + slatr*TPSLRatio
            self.buy(sl=sl1, tp=tp1, size=self.mysize)

        elif self.signal1==1 and len(self.trades)==0:
            sl1 = self.data.Close[-1] + slatr
            tp1 = self.data.Close[-1] - slatr*TPSLRatio
            self.sell(sl=sl1, tp=tp1, size=self.mysize)

In [43]:

bt = Backtest(scalping.df, MyStrat, cash=250, margin=1/30)
bt.run()

C:\Users\S.Eyvazi\AppData\Local\Temp\ipykernel_10248\2894922998.py:1: UserWarning: Data index is not datetime. Assuming simple periods, but `pd.DateTimeIndex` is advised.
  bt = Backtest(scalping.df, MyStrat, cash=250, margin=1/30)


Start                                     0.0
End                                   29998.0
Duration                              29998.0
Exposure Time [%]                   46.301543
Equity Final [$]                    399.19517
Equity Peak [$]                    416.460472
Return [%]                          59.678068
Buy & Hold Return [%]                1.967449
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -24.394674
Avg. Drawdown [%]                   -1.262427
Max. Drawdown Duration                 9571.0
Avg. Drawdown Duration             189.219178
# Trades                               1188.0
Win Rate [%]                        51.178451
Best Trade [%]                        0.45531
Worst Trade [%]                     -0.465502
Avg. Trade [%]                    

In [25]:
import numpy as np 
a = np.arange(0,3, .1)
a




array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2,
       1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5,
       2.6, 2.7, 2.8, 2.9])

In [130]:
df

,Gmt time,<DATE>,<TIME>,Open,High,Low,Close,<TICKVOL>,<VOL>,<SPREAD>,...,EMA_fast,RSI,ATR,BBL_15_1.5,BBM_15_1.5,BBU_15_1.5,BBB_15_1.5,BBP_15_1.5,EMASignal,TotalSignal
0,2022-05-09 00:05:00,2022.05.09,00:05:00,1.05400,1.05425,1.05395,1.05407,34,0,52,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,0
1,2022-05-09 00:10:00,2022.05.09,00:10:00,1.05407,1.05425,1.05403,1.05417,20,0,65,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,2022-05-09 00:15:00,2022.05.09,00:15:00,1.05418,1.05424,1.05389,1.05420,37,0,13,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
3,2022-05-09 00:20:00,2022.05.09,00:20:00,1.05390,1.05424,1.05385,1.05420,26,0,25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,2022-05-09 00:25:00,2022.05.09,00:25:00,1.05391,1.05421,1.05387,1.05399,32,0,32,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30202,2022-09-30 23:30:00,2022.09.30,23:30:00,0.98080,0.98080,0.98022,0.98022,214,0,3,...,0.980138,47.769485,0.000448,0.979621,0.980239,0.980856,0.125924,0.484877,2,0
30203,2022-09-30 23:35:00,2022.09.30,23:35:00,0.98023,0.98027,0.97993,0.98011,159,0,7,...,0.980136,45.531832,0.000433,0.979720,0.980276,0.980832,0.113433,0.350714,2,0
30204,2022-09-30 23:40:00,2022.09.30,23:40:00,0.98012,0.98036,0.97998,0.98021,101,0,3,...,0.980141,47.992608,0.000425,0.979768,0.980299,0.980831,0.108454,0.415975,2,0
30205,2022-09-30 23:45:00,2022.09.30,23:45:00,0.98017,0.98029,0.98013,0.98025,88,0,3,...,0.980148,49.016320,0.000387,0.979893,0.980343,0.980794,0.091945,0.396455,2,0
